In [16]:
from __future__ import print_function

# ignore some Keras warnings regarding deprecations and model saving 
import warnings
warnings.filterwarnings('ignore')

from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense
import pickle

from helpers import *


In [2]:
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples =70000 # Number of samples to train on.
# Path to the datatxt file on disk.
data_path = 'fra.txt'


In [18]:
def load_single_preprocessed(data_path):
    input_phrases, _ = read_data(data_path)
    return text_preprocess(input_phrases)

In [19]:
all_phrases = load_single_preprocessed(data_path)
input_token_index = token_index(all_phrases)
num_encoder_tokens = len(input_token_index)

target_token_index = {'\t': num_encoder_tokens, 
                      '\n': num_encoder_tokens+1, 
                      **input_token_index}
max_encoder_seq_length = max([len(txt) for txt in input_phrases])
max_decoder_seq_length = max_encoder_seq_length + 5

In [ ]:
input_phrases, target_phrases = read_data(data_path)
input_phrases = text_preprocess(input_phrases)
target_phrases = text_preprocess(target_phrases)
# keep only `num_samples` examples. 
# Note: we're starting from the 2nd example, its not important
input_phrases, target_phrases = input_phrases[1:num_samples+1], target_phrases[1:num_samples+1]
# Wrap the target phrases with delimiters signifying the start end end of the sequence
target_phrases = wrap_with_delims(target_phrases)
print("Training on %d selected phrases" % len(input_phrases))

In [6]:


# Preprocess input and target
input_phrases = text_preprocess(input_phrases)

input_phrases, target_phrases = misspelling_gen(input_phrases, 0.07, 3)
target_phrases = wrap_with_delims(target_phrases)

In [7]:
list(zip(input_phrases[:5], target_phrases[:5]))

[('run!', '\trun!\n'),
 ('run!', '\trun!\n'),
 ('wow!', '\twow!\n'),
 ('fire!', '\tfire!\n'),
 ('help!', '\thelp!\n')]

In [8]:
# a dict of the index of each token

print(token_index(['abc', 'cde!']))

{'!': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5}


In [9]:
input_token_index = token_index(input_phrases)
target_token_index = token_index(target_phrases)
num_encoder_tokens = len(input_token_index)
num_decoder_tokens = len(target_token_index)
max_encoder_seq_length = max([len(txt) for txt in input_phrases])
max_decoder_seq_length = max([len(txt) for txt in target_phrases])
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of unique input tokens: 53
Number of unique output tokens: 55
Max sequence length for inputs: 29
Max sequence length for outputs: 30


In [11]:
encoder_input_data = vectorize_batch(input_phrases, input_token_index, max_encoder_seq_length)
decoder_input_data = vectorize_batch(target_phrases, target_token_index, max_decoder_seq_length)
# same as decoder input data, but offset by one
decoder_target_data = vectorize_batch(target_phrases, target_token_index, max_decoder_seq_length, True)



In [12]:
def models(num_encoder_tokens, num_decoder_tokens, latent_dim):
    # Define an input sequence and process it.
    encoder_inputs = Input(shape=(None, num_encoder_tokens), name='encoder_inputs')
    encoder = LSTM(latent_dim, return_state=True, name='encoder')
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    # We discard `encoder_outputs` and only keep the states.
    encoder_states = [state_h, state_c]

    # Set up the decoder, using `encoder_states` as initial state.
    decoder_inputs = Input(shape=(None, num_decoder_tokens), name='decoder_inputs')
    # We set up our decoder to return full output sequences,
    # and to return internal states as well. We don't use the
    # return states in the training model, but we will use them in inference.
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, 
                        name='decoder_lstm')
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                         initial_state=encoder_states)
    decoder_dense = Dense(num_decoder_tokens, activation='softmax', 
                          name='decoder_dense')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model that will turn
    # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # Next: inference mode (sampling).
    # Here's the drill:
    # 1) encode input and retrieve initial decoder state
    # 2) run one step of decoder with this initial state
    # and a "start of sequence" token as target.
    # Output will be the next target token
    # 3) Repeat with the current target token and current states

    # Define sampling models
    encoder_model = Model(encoder_inputs, encoder_states)

    decoder_state_input_h = Input(shape=(latent_dim,), name='decoder_input_h')
    decoder_state_input_c = Input(shape=(latent_dim,), name='decoder_input_c')
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return model, encoder_model, decoder_model


In [13]:
model, encoder_model, decoder_model = models(num_encoder_tokens, num_decoder_tokens, latent_dim)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [14]:
history = model.fit([encoder_input_data, decoder_input_data], 
                   decoder_target_data,
                   batch_size=batch_size, epochs=epochs, 
                   validation_split=0.1)

Train on 252000 samples, validate on 28000 samples
Epoch 1/10
134400/252000 [===============>..............] - ETA: 9:07 - loss: 1.1635

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
def plot_history(history):
    """credit: https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/"""
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
plot_history(history)

In [ ]:
# Save model


In [ ]:
model.save('training.h5')
encoder_model.save('encoder.h5')
decoder_model.save('decoder.h5')

model_metadata = { 'input_token_index': input_token_index, 
                   'target_token_index': target_token_index,
                   'max_encoder_seq_length': max_encoder_seq_length }

with open('model_metadata.pickle', 'wb') as f:
    pickle.dump(model_metadata, f)

In [ ]:
model = load_model('training.h5')
encoder_model = load_model('encoder.h5')
decoder_model = load_model('decoder.h5')

with open('model_metadata.pickle', 'rb') as f:
    model_metadata = pickle.load(f)

input_token_index = model_metadata['input_token_index']
target_token_index = model_metadata['target_token_index']
max_encoder_seq_length = model_metadata['max_encoder_seq_length']

In [15]:
model.evaluate([encoder_input_data, decoder_input_data], 
                   decoder_target_data,)

 17376/280000 [>.............................] - ETA: 9:44

KeyboardInterrupt: 

In [13]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = {v: k for k, v in input_token_index.items()}
reverse_target_char_index = {v: k for k, v in target_token_index.items()} 

In [16]:
[l.name for l in decoder_model.layers]

['decoder_inputs',
 'decoder_input_h',
 'decoder_input_c',
 'decoder_lstm',
 'decoder_dense']

In [17]:
input_phrases[:25]

['run!',
 'run!',
 'wow!',
 'fire!',
 'help!',
 'jump',
 'stop!',
 'stop!',
 'stop!',
 'wait!',
 'wait!',
 'i see',
 'i try',
 'i won!',
 'i won!',
 'oh no!',
 'attack!',
 'attack!',
 'cheers!',
 'cheers!',
 'cheers!',
 'cheers!',
 'get up',
 'got it!',
 'got it!']

In [18]:
def translate(phrase, encoder_model, decoder_model, 
              input_token_index, target_token_index):
    vect = vectorize_phrase(phrase, input_token_index,
                            max_encoder_seq_length)
    decoded = decode_sequence(vect,target_token_index, 
                        encoder_model, decoder_model)
    return decoded[:-1]

translator = lambda p: translate(p, encoder_model, decoder_model,
                                input_token_index, target_token_index)

In [19]:
# find max encoder seq legth
#max_encoder_seq_length = encoder_model.get_layer('encoder_inputs').input_shape[-1]
phrases = ['fire', 'stp', 'comein', 'get ot', 'i cant go',
          'im sorry', 'h is busi', 'hes drunk']

[translator(phrase) for phrase in phrases]

['fire out',
 'stop',
 'come in',
 'get out',
 'i can go',
 "i'm sor",
 'he is busy',
 "he's drunk"]

In [28]:
test_phrases = input_phrases[20:1020]
misspelled_test_phrases = [add_noise_to_string(p, .1) for p in test_phrases]

pairs = zip(misspelled_test_phrases, test_phrases)
errors = list(filter(lambda p: translator(p[0]) != p[1], pairs))
len(errors)/float(len(test_phrases))

0.232

In [12]:
input_phrases, _ = read_data(data_path)
input_phrases = text_preprocess(input_phrases)

encoder_vectorizer = lambda b: vectorize_batch(b, input_token_index,
                                               max_encoder_seq_length)
decoder_vectorizer = lambda b, offset: vectorize_batch(b, target_token_index,
                                                       max_decoder_seq_length, offset) 

def vectorized_gen(phrases, batch_size,encoder_vectorizer, decoder_vectorizer):
    for i in range(0, len(phrases), batch_size):
        input_phrases, target_phrases =  misspelling_gen(phrases[i:i+batch_size], 0.07, 3)
        print(len(input_phrases))
        print(len(target_phrases))
        encoder_input_data = encoder_vectorizer(input_phrases)
        decoder_input_data = decoder_vectorizer(target_phrases, False)
        decoder_target_data = decoder_vectorizer(target_phrases, True)
        max_encoder_seq_length = max([len(txt) for txt in input_phrases])
        
        yield ([encoder_input_data, decoder_input_data], decoder_target_data)
        print('yielded')

training_generator = vectorized_gen(input_phrases, num_samples, encoder_vectorizer, decoder_vectorizer)


In [14]:
input_token_index = token_index(input_phrases)
target_token_index = token_index(target_phrases)
num_encoder_tokens = len(input_token_index)
num_decoder_tokens = len(target_token_index)
max_encoder_seq_length = max([len(txt) for txt in input_phrases])
max_decoder_seq_length = max([len(txt) for txt in target_phrases])

In [15]:
for _ in range(5):
    spell_gen = lambda : misspelling_gen(input_phrases, 0.07, 3)
    X, Y = next(training_generator)
    model.fit(X, Y,
          batch_size=batch_size,
          epochs=5,
          validation_split=0.1)


240000
240000


MemoryError: 